# Constantes et algorithme PSLQ

$I=\int_0^{\pi/2} \ln(\sin x)\,dx$


1-Évaluation de  I à 50, 100 puis 200 décimales

In [2]:

from mpmath import mp, pi, quad,zeta,ln,pslq
def calculer_I(dps):
    mp.dps = dps  # Définit la précision en décimales
    # Définition de la fonction à intégrer
    fonction = lambda x: mp.log(mp.sin(x))
    # Calcul de l'intégrale avec quad (méthode adaptative)
    I,erreur_estimee = quad(fonction, [0, pi/2], error=True)
    return I,erreur_estimee
precisions=[50,100,200]
I_list=[]
error_list=[]
for dps in precisions:
    I,erreur_estimee = calculer_I(dps)
    I_list.append(I)
    error_list.append(erreur_estimee)
for dps, I, err in zip(precisions, I_list, error_list):
    print("="*40)
    print(f"Précision : {dps} décimales")
    print(f"I = {I}")
    print(f"Erreur estimée : {err}")
print("="*40)



Précision : 50 décimales
I = -1.08879304515180106525034444911880697366929185018464418410171727866266177123169161860787420487291456006980048322252751413880228308073139231737513910047709941864013671875
Erreur estimée : 9.9999999999999999999999999999999999999999999999999999999964114613314089819853693519360754690685289495859954004249822465351165175435147403994803629092751093742546268920557952792944196062538733022649421e-100
Précision : 100 décimales
I = -1.0887930451518010652503444491188069736692918501846431471628976265971542745883709932151644808053315125281176720529077047881424588474564060205364116419322006978512554967638271351018526362094436650382873
Erreur estimée : 1.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000030857865545825071101585216694835384322115591759318743441532569405196911828224785339650684263e-190
Précision : 200 décimales
I = -1.08879304515180106525034444911880697366929185018464314716289762659715427458837099321516

2- Application de  pslq au vecteur $(I,\, \zeta(2),\, \ln^2 2,\, \ln 2)$.

In [3]:
for dps, I in zip(precisions, I_list):
    mp.dps = dps  # Assure la même précision pour toutes les constantes
    v = [I, zeta(2),ln(2)**2,ln(2)]
    relation = pslq(v)
    print("="*40)
    print(f"Précision : {dps} décimales")
    print(f"Relation PSLQ trouvée : {relation}")
print("="*40)



Précision : 50 décimales
Relation PSLQ trouvée : None
Précision : 100 décimales
Relation PSLQ trouvée : None
Précision : 200 décimales
Relation PSLQ trouvée : None


3. Détection d une relation rationnelle et verification à haute précision.

In [4]:

mp.dps = 1000 # haute precision
I = quad(lambda x: mp.log(mp.sin(x)), [0, pi/2])
pi_ln2 = pi * ln(2)
vecteur = [I, pi_ln2]
relation = pslq(vecteur)
print("Relation PSLQ trouvée :", relation)

if relation:
    # Recalcule tout à la même précision, juste avant la vérification
    test = relation[0]*I + relation[1]*pi_ln2
    print("Vérification :", test)



Relation PSLQ trouvée : [2, 1]
Vérification : 4.7569042372799272292826261945299935942145799977380388050834927630064346701657070079429427664755683246849537712984683349207413092748631052862500756726490946686494978701705430330426733260383248645632712855279708529971184598047531025491969687544134857367097493516171063361138993484848123509575321251976166747720814473772845621323503771840597378377687478160119999908101108374592122070113324314847378906118486931816324635787718427538862745875498744891161780511520891114160243965824248740521129791779933723860327105249345622742224046031257478962406277428423406460177025729433776402595331375827885544705223442240836135506341982285090171345667748851293882046994163897144274967352430486582313228508525101948385996623605308641775516534024824619994332546777688281239988118431870388690346801093886837763117296189391042029947048568700587238813604823236009712481196861839697626996452123474716921387693371417005534509860188109727311220210025723600597182361788738077463

4-Étude de la stabilité de PSLQ quand la précision varie.



In [6]:


# Liste des précisions à tester 
precisions = [ 80, 100,300, 400, 500,600,700]

print("Précision | Relation PSLQ trouvée           | Résidu")
print("-"*70)
for dps in precisions:
    mp.dps = dps
    # Calcul de I à la précision courante
    I = quad(lambda x: mp.log(mp.sin(x)), [0, pi/2])
    pi_ln2 = pi * ln(2)
    vecteur = [I, pi_ln2]
    relation = pslq(vecteur)
    if relation:
        residu = relation[0]*I + relation[1]*pi_ln2
    else:
        residu = "Aucune"
    print(f"{dps:<9} | {str(relation):<30} | {residu}")


Précision | Relation PSLQ trouvée           | Résidu
----------------------------------------------------------------------
80        | [-2, -1]                       | 0.0
100       | [-2, -1]                       | 0.0
300       | [2, 1]                         | 3.73305447401287551596035817889526867846836578548683209848685735918386764390310253781776130839152440943837995972129697049686195008541612957936608326881572302493764266455330060109598030394360732604440196318506045247296205005918373516322071308450166041524279351541770592447787925691464383688807065164177e-301
400       | [-2, -1]                       | 0.0
500       | [-2, -1]                       | 0.0
600       | [-2, -1]                       | 5.57427828237901867236876511665847826934982639533144871465183566687890486618001114482223280051186424388196101579357127720717075351807253524805298612769474359110496681107859028380653383896954203126275748079486230220200386374218945498456362767450540846607122737504949726313989833476975

5-Proposer une conjecture fermée pour I(dans le rapport Latex)